In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('finalMergedFile.csv')

In [3]:
df = df.drop_duplicates() 
df.shape

(321195, 15)

In [4]:
df.duplicated().value_counts() # To check if the code worked 

False    321195
dtype: int64

In [39]:
df.head()

,client_id,visitor_id,visit_id,process_step,date_time,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,Variation
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0,Test
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0,Test
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0,Test
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0,Test
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0,Test


In [ ]:
df['process_step'].unique() # This is the complete registration process

#### analysing test data , how many people started and how many finished 

In [40]:
test = df[df['Variation']=='Test']
test = test.reset_index(drop=True)
test.shape

(176641, 14)

In [41]:
test_started = test[test['process_step']=='start'].reset_index(drop=True)
# test_started.head()

# number of people who started the test is 
print(test_started['client_id'].nunique())

26672


In [42]:
# number of unique ids in the dataframe 
print(test['client_id'].nunique())

26961


In [43]:
# this difference means that there are a lot of client ids that did not start but
# present in this data set. THey would be removed from analysis as they are faulty onservations

In [44]:
ids = set(test['client_id'].unique())-set(test_started['client_id'].unique())
ids = list(ids)
print(len(ids))
ids[:5]

289


[3580416, 6225921, 5899779, 7910915, 9228292]

In [45]:
test = test[test['client_id'].isin(test_started['client_id'].unique())]
test.shape  # clean test

(175666, 14)

In [46]:
test[test['process_step']=='start']['client_id'].nunique()

26672

In [47]:
print(len(test[test['process_step']=='start']['client_id']))
print('since this number is different than unique start values, therefore there should be some customers that repeated the process')

55753
since this number is different than unique start values, therefore there should be some customers that repeated the process


In [48]:
test[test['process_step']=='confirm']['client_id'].nunique()

18407

##### In test variation, 26672 clients started the test and 18407 finished 

In [49]:
# Now we will follow a similar strategy for control data

In [50]:
control = df[df['Variation']=='Control'].reset_index(drop=True)
control.shape

(140482, 14)

In [51]:
ids_list = set(control['client_id'].unique())-set(control[control['process_step']=='start']['client_id'].unique())
ids_list = list(ids_list)
print(len(ids_list))
ids_list[:5]

135


[5907969, 4424716, 6338574, 89108, 1363990]

In [52]:
keep_list = control[control['process_step']=='start']['client_id'].unique()
control = control[control['client_id'].isin(keep_list)]
control.shape

(139996, 14)

In [53]:
len(control[control['process_step']=='confirm'])

17168

In [54]:
control[control['process_step']=='confirm']['client_id'].nunique()

15312

In [55]:
len(control[control['process_step']=='start'])

45361

In [57]:
control['client_id'].nunique()

23391

##### In control variation, 23391 clients started the test and 15312 finished

In [ ]:
# Control : Percent of clients completed the process

In [58]:
round(15312/23391,4)

0.6546

In [59]:
# Test : Percent of clients completed the process

In [60]:
round(18407/26672,4)

0.6901

In [ ]:
test_uniques = test.drop_duplicates(subset=['client_id', 'process_step'], keep='last').sort_values(by=['client_id', 'date_time'])
# test_uniques

In [ ]:
control_uniques = control.drop_duplicates(subset=['client_id', 'process_step'], keep='last').sort_values(by=['client_id', 'date_time'])
# control_uniques

In [ ]:
test_uniques['process_step'].value_counts()

In [ ]:
control_uniques['process_step'].value_counts()

In [ ]:
# With this we can see the number of drop outs at each step

In [ ]:
pd.concat([test_uniques, control_uniques], axis=0).to_csv('tableau_file.csv')

In [ ]:
tableau = pd.concat([test_uniques, control_uniques], axis=0).reset_index(drop=True)
tableau.shape

In [ ]:
tableau[tableau['client_id']==5848988]

In [ ]:
tableau['clnt_age'].describe()

In [ ]:
def step_counter(df):
    start_check = []
    step1 = []
    step2 = []
    step3 = []
    confirm = []
    clients = df['client_id'].unique()
    for client in clients:
        test_df = df[df['client_id']==client]
        process_steps_followed = list(set(test_df['process_step']))
        if 'confirm' in process_steps_followed:
            confirm.append(client)
        elif 'step_3' in process_steps_followed:
            step3.append(client)
        elif 'step_2' in process_steps_followed:
            step2.append(client)   
        elif 'step_1' in process_steps_followed:
            step1.append(client)
        else:
            start_check.append(client)
    return start_check, step1, step2, step3, confirm

In [ ]:
start_check_test, step1_test, step2_test, step3_test, confirm_test = step_counter(test)


In [ ]:
total_test = test['client_id'].nunique()
len(confirm_test)/total_test

In [ ]:
len(step3_test)

In [ ]:
len(step3_test)/total_test

In [ ]:
len(step2_test)

In [ ]:
len(step2_test)/total_test

In [ ]:
len(step1_test)

In [ ]:
len(step1_test)/total_test

In [ ]:
len(start_check_test)

In [ ]:
len(start_check_test)/total_test

In [ ]:
total_test = len(confirm_test)+len(step3_test)+len(step2_test)+len(step1_test)+len(start_check_test)
total_test

In [ ]:
start_check_control, step1_control, step2_control, step3_control, confirm_control = step_counter(control)


In [ ]:
len(confirm_control)

In [ ]:
total_control = control['client_id'].nunique()
len(confirm_control)/total_control

In [ ]:
len(step3_control)

In [ ]:
len(step3_control)/total_control

In [ ]:
len(step2_control)

In [ ]:
len(step2_control)/total_control

In [ ]:
len(step1_control)

In [ ]:
len(step1_control)/total_control

In [ ]:
len(start_check_control)

In [ ]:
len(start_check_control)/total_control

In [ ]:
total_control = len(confirm_control)+len(step3_control)+len(step2_control)+len(step1_control)+len(start_check_control)
total_control

In [ ]:
# df[df['client_id']==836976]

In [ ]:
counts_test = pd.DataFrame(test.groupby(['client_id', 'process_step']).agg({'Variation': 'count'})).reset_index()
# counts_test

In [ ]:
counts_test_start = counts_test[counts_test['process_step']=='start']
counts_test_start = counts_test_start[counts_test_start['Variation']>1]
counts_test_start = counts_test_start.reset_index().drop(['index'], axis=1)

counts_test_step1 = counts_test[counts_test['process_step']=='step_1']
counts_test_step1 = counts_test_step1[counts_test_step1['Variation']>1]
counts_test_step1 = counts_test_step1.reset_index().drop(['index'], axis=1)

counts_test_step2 = counts_test[counts_test['process_step']=='step_2']
counts_test_step2 = counts_test_step2[counts_test_step2['Variation']>1]
counts_test_step2 = counts_test_step2.reset_index().drop(['index'], axis=1)

counts_test_step3 = counts_test[counts_test['process_step']=='step_3']
counts_test_step3 = counts_test_step3[counts_test_step3['Variation']>1]
counts_test_step3 = counts_test_step3.reset_index().drop(['index'], axis=1)

counts_test_confirm = counts_test[counts_test['process_step']=='comfirm']
counts_test_confirm = counts_test_confirm[counts_test_confirm['Variation']>1]
counts_test_confirm = counts_test_confirm.reset_index().drop(['index'], axis=1)

In [ ]:
counts_test_start.head()

In [ ]:
counts_control = pd.DataFrame(control.groupby(['client_id', 'process_step']).agg({'Variation': 'count'})).reset_index()
counts_control.head()

In [ ]:
counts_control_start = counts_control[counts_control['process_step']=='start']
counts_control_start = counts_control_start[counts_control_start['Variation']>1]
counts_control_start = counts_control_start.reset_index().drop(['index'], axis=1)

counts_control_step1 = counts_control[counts_control['process_step']=='step_1']
counts_control_step1 = counts_control_step1[counts_control_step1['Variation']>1]
counts_control_step1 = counts_control_step1.reset_index().drop(['index'], axis=1)

counts_control_step2 = counts_control[counts_control['process_step']=='step_2']
counts_control_step2 = counts_control_step2[counts_control_step2['Variation']>1]
counts_control_step2 = counts_control_step2.reset_index().drop(['index'], axis=1)

counts_control_step3 = counts_control[counts_control['process_step']=='step_3']
counts_control_step3 = counts_control_step3[counts_control_step3['Variation']>1]
counts_control_step3 = counts_control_step3.reset_index().drop(['index'], axis=1)

counts_control_confirm = counts_control[counts_control['process_step']=='comfirm']
counts_control_confirm = counts_control_confirm[counts_control_confirm['Variation']>1]
counts_control_confirm = counts_control_confirm.reset_index().drop(['index'], axis=1)
